# Cassandra

## Conexión con Cassandra y borrado de datos

In [1]:
%load_ext cql

In [4]:
%%cql
DROP KEYSPACE black;

'No results.'

In [5]:
%%cql
CREATE KEYSPACE black 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [6]:
%cql USE black;

'No results.'

## Creacción de las tablas

En **Cassandra** vamos a crear 2 tablas principales, una donde almacenamos los movimientos en formato desnormalizado, y otra que nos servirá para almacenar los importes acumulados por cliente.

También tenemos una tabla auxiliar, creada gracias a la funcionalidad que nos da la base de datos de VISTAS MATERIALIZADAS, que vamos a utilizar para facilitar otros patrones distintos de acceso a la información.

En la tabla que almacena los movimientos, como no tenemos una PK clara, vamos a utilizar un tipo UUID (marca temporal única) donde guardaremos un TIMESTAMP con la hora de creación del registro

<br><br> 

<img src="images/Modelo%20Cassandra.png",width=800,height=500>

<br><br> 

In [7]:
%%cql 
CREATE TABLE acum_movimientos_nombre (
    nombre          text,
    importe         counter,
    PRIMARY KEY (nombre)
)

'No results.'

In [8]:
%%cql 
CREATE TABLE movimientos (
    uid             uuid,
    fecha           date,
    hora            int,
    minuto          int,
    importe         decimal,
    comercio        text,
    actividad_completa text,
    actividad       text,
    nombre          text,
    funcion         text,
    organizacion    text,
    PRIMARY KEY ((nombre), importe, fecha, hora, minuto)
)
WITH CLUSTERING ORDER BY (importe DESC);

'No results.'

En la vista materializada no es necesario insertar datos ya que se ocupa la propia base de datos

In [9]:
%%cql
CREATE MATERIALIZED VIEW vm_movimientos_by_actividad AS
   SELECT * FROM movimientos
   WHERE actividad IS NOT NULL 
         and importe IS NOT NULL 
         and fecha IS NOT NULL
         and hora IS NOT NULL 
         and minuto IS NOT NULL 
         and nombre IS NOT NULL
   PRIMARY KEY (actividad, importe, fecha,hora,minuto,nombre)
   WITH CLUSTERING ORDER BY (importe desc)

'No results.'

## Carga de datos en pandas

Cargamos los datos en formato Excel y los desnormalizamos ...

In [10]:
import pandas as pd
df_mov = pd.read_excel("../../data/black.xlsx", sheetname= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheetname= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
id_miembro            7624 non-null int64
fecha                 7624 non-null datetime64[ns]
minuto                7624 non-null int64
hora                  7624 non-null int64
importe               7624 non-null float64
comercio              6864 non-null object
actividad_completa    7622 non-null object
actividad             7622 non-null object
nombre                7624 non-null object
funcion               7624 non-null object
organizacion          6056 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


### Carga de datos en Cassandra

In [11]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('black')

A destacar varios aspectos:
- El tipo de las fechas. Hay que dejarlo con el tipo **Date** de Python para no tener problemas (El tipo de la columna es también Date)
- Los nulos en Pandas son del tipo NumPy.nan, por lo hay convertirlos a None para que la inserción en Cassandra sea correcta
- En la tabla de importes acumulados guardamos el importe en un dato de tipo **counter**, que tiene un tipo entero, por lo que multiplicamos el dato por 100 para poder almacenarlo correctamete

In [12]:
df.ix[df.comercio.isnull(), 'comercio'] = None
df.ix[df.organizacion.isnull(), 'organizacion'] = None
df.ix[df.actividad.isnull(), 'actividad'] = None
df.ix[df.actividad_completa.isnull(), 'actividad_completa'] = None

In [13]:
import dateutil

def insert_movientos(df):
    
    sql_insert = """
INSERT INTO movimientos (
uid,
importe,
actividad,
fecha,
actividad_completa,
comercio,
funcion,
hora,
minuto,
organizacion,
nombre
) VALUES (now(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

    for index in df.index:
        fecha = df.ix[index, 'fecha']
        
        data = [
            df.ix[index, "importe"],
            df.ix[index, "actividad"],
            fecha.to_datetime().date(),
            df.ix[index, "actividad_completa"],
            df.ix[index, "comercio"],
            df.ix[index, "funcion"],
            df.ix[index, "hora"],
            df.ix[index, "minuto"],
            df.ix[index, "organizacion"],
            df.ix[index, "nombre"]
        ]
        
        session.execute(sql_insert, data)
        
        importe_int = int(round(df.ix[index, "importe"] * 100))        
        session.execute("UPDATE acum_movimientos_nombre SET importe = importe + %s WHERE nombre = %s", 
                        [importe_int, 
                         df.ix[index, "nombre"]]
        )

Ya podemos insertar los datos ...

In [14]:
insert_movientos(df)

## Consulta de datos

Función de utilidad que realiza un query en Cassandra y devuelve un DataFrame de Pandas

In [15]:
def execute_query(sql):
    rows = session.execute(sql)
    return pd.DataFrame(list(rows))

### Los 10 movimientos mas caros

Para resolver esta query con la base de datos tendríamos que tener un PARTITION KEY única para todos los registros

In [16]:
sql = """
SELECT nombre, fecha, actividad_completa, importe
FROM MOVIMIENTOS
"""
df = execute_query(sql)

In [17]:
df.sort_values('importe', ascending = False).head(10)

,nombre,fecha,actividad_completa,importe
4737,Ildefonso José Sánchez Barcoj,2006-02-14,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),11000.0
4738,Ildefonso José Sánchez Barcoj,2008-11-19,EL CORTE INGLES,6593.2
5684,Matías Amat Roca,2010-11-30,AGENCIAS DE VIAJES,6519.12
3407,María Carmen Cafranga Cavestany,2011-02-14,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,5500.0
4739,Ildefonso José Sánchez Barcoj,2004-01-08,AGENCIAS DE VIAJES,5283.33
6206,Ricardo Morado Iglesias,2007-02-01,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),5000.0
1446,Estanislao Rodríguez-Ponga Salamanca,2009-12-25,EL CORTE INGLES,5000.0
4296,Ramón Martínez Vilches,2010-07-07,AGENCIAS DE VIAJES,4955.0
6207,Ricardo Morado Iglesias,2007-06-25,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),4500.0
4740,Ildefonso José Sánchez Barcoj,2010-03-25,EL CORTE INGLES,4320.5


### Los movimientos de una persona concreta

Se resuelve la query con la base de datos, ya que la PARTITION KEY es el campo **nombre**, y los registros están ordenados por importe

In [18]:
sql = """
select nombre, fecha, actividad_completa, importe
from movimientos
where nombre = 'Javier de Miguel Sánchez'
limit 10
"""
execute_query(sql)

,nombre,fecha,actividad_completa,importe
0,Javier de Miguel Sánchez,2010-01-12,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",383.0
1,Javier de Miguel Sánchez,2009-05-30,CONFECCION TEXTIL EN GENERAL,350.0
2,Javier de Miguel Sánchez,2010-01-12,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",345.0
3,Javier de Miguel Sánchez,2009-03-27,HIPERCOR SUPERMERCADOS EL CORTE INGLES,268.5
4,Javier de Miguel Sánchez,2010-01-19,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",260.72
5,Javier de Miguel Sánchez,2004-04-21,JOYERIAS Y RELOJERIAS,240.0
6,Javier de Miguel Sánchez,2009-05-29,HIPERCOR SUPERMERCADOS EL CORTE INGLES,236.05
7,Javier de Miguel Sánchez,2008-12-29,HIPERCOR SUPERMERCADOS EL CORTE INGLES,236.0
8,Javier de Miguel Sánchez,2006-08-30,EL CORTE INGLES,233.39
9,Javier de Miguel Sánchez,2004-07-14,RESTAURANTES RESTO,213.25


### Las 10 personas que mas han gastado

Esta consulta se pueder resolver mediante el acumulado que habíamos creado ad-doc. Observa que el importe es entero, por lo que hay que dividirlo por 100 para convertirlo a decimal

In [19]:
sql = """
select *
from acum_movimientos_nombre
"""
df = execute_query(sql)

In [20]:
df.assign(importe = df.importe / 100).sort_values('importe', ascending = False).head(10)

,nombre,importe
57,Ildefonso José Sánchez Barcoj,64590.50
73,Ricardo Morado Iglesias,47292.08
72,José Antonio Moral Santín,41207.26
59,Juan Manuel Astorqui Portera,39946.33
67,Matías Amat Roca,39820.26
36,Carlos María Martínez Martínez,39473.01
55,Miguel Blesa de la Parra,35475.15
4,Ramón Ferraz Ricarte,30612.27
33,Maria Mercedes de la Merced Monge,26030.22
30,Mariano Pérez Claver,25113.06


### Los 10 movimientos mas caros por actividad

Esta consulta se pueder resolver mediante la vista materializa que hemos creado ad-hoc para resolver esta consulta

In [21]:
sql = """
select nombre, fecha, actividad, actividad_completa, importe
from vm_movimientos_by_actividad
where actividad = 'HOGAR'
limit 10
"""
execute_query(sql)

,nombre,fecha,actividad,actividad_completa,importe
0,Miguel Ángel Araujo Serrano,2007-10-14,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2150.0
1,Mariano Pérez Claver,2003-04-11,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2072.0
2,Carlos María Martínez Martínez,2007-05-01,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1510.0
3,Ricardo Romero de Tejada y Picatoste,2009-08-06,HOGAR,"TAPICERIAS,ALFOMBRAS",1339.94
4,Rafael Spottorno Díaz Caro,2003-08-10,HOGAR,MIRO ESTABLECIMIENTOS,1198.0
5,Ricardo Romero de Tejada y Picatoste,2008-02-14,HOGAR,FLORES Y PLANTAS,1000.0
6,Ricardo Romero de Tejada y Picatoste,2009-04-06,HOGAR,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",1000.0
7,Alejandro Couceiro Ojeda,2005-05-09,HOGAR,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",948.0
8,Enrique de la Torre Martínez,2003-01-02,HOGAR,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",912.0
9,Pablo Abejas Juárez,2011-03-27,HOGAR,IKEA,835.65
